In [ ]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import json
from time import sleep
import os
import random
from datetime import datetime, timedelta
import geopandas as gpd
from shapely.geometry import Point
import time
import numpy as np
from matplotlib.colors import TwoSlopeNorm, Normalize
from matplotlib.cm import ScalarMappable
import matplotlib.colors as mcolors
import folium
from branca.colormap import linear
import statsmodels.api as sm
import statsmodels.formula.api as smf
import branca.colormap as cm
import statsmodels.formula.api as smf

## Load matched pairs dataframe

In [ ]:
filtered_df2 = pd.read_csv('CityMatchedPOIsCleanedforDID.csv')

In [ ]:
long_df = pd.DataFrame({
    'poi_id': filtered_df2['Original POI ID'].tolist() * 2 + filtered_df2['POI ID'].tolist() * 2,
    'group': ['original'] * len(filtered_df2) * 2 + ['similar'] * len(filtered_df2) * 2,
    'time': ['pre'] * len(filtered_df2) + ['post'] * len(filtered_df2) + ['pre'] * len(filtered_df2) + ['post'] * len(filtered_df2),
    'visits': filtered_df2['Original Avg Pre Open'].tolist() + filtered_df2['Original Avg Post Open'].tolist() + filtered_df2['Avg Pre Open'].tolist() + filtered_df2['Avg Post Open'].tolist(),
    'city': filtered_df2['City'].tolist() *2 + filtered_df2['City'].tolist() * 2,
    'category': filtered_df2['Group'].tolist() * 2 + filtered_df2['Group'].tolist() * 2,
    'Similar_POI_Zip_Code': filtered_df2['Zip_Code'].tolist() * 2 + filtered_df2['Zip_Code'].tolist() * 2,
    'distance_group': filtered_df2['distance_group'].tolist() * 2 + filtered_df2['distance_group'].tolist() * 2,
    'year': filtered_df2['Original POI Open Year'].tolist() * 4
})

In [ ]:
long_df['groupbinary']= long_df['group'].apply(lambda x:1 if x=='original' else 0)
long_df['timebinary']= long_df['time'].apply(lambda x:1 if x=='post' else 0)

In [ ]:
long_df

In [ ]:
long_df['groupbinary']= long_df['group'].apply(lambda x:1 if x=='original' else 0)
long_df['timebinary']= long_df['time'].apply(lambda x:1 if x=='post' else 0)

## Vanila DID algorithm

In [ ]:
model = smf.ols('visits ~ (groupbinary) : (timebinary) + C(year) +  C(poi_id)', data=long_df).fit()
print(model.summary())

## Heterogeneous treatment effects

In [ ]:
distance_groups = ['0-100m', '100-200m', '200-300m', '300-400m']

In [ ]:
for group in distance_groups:
    long_df[f'distance_group_{group.replace("-", "_").replace("m", "")}'] = (long_df['distance_group'] == group).astype(int)

In [ ]:
poi_categories = ['Apparel', 'Beauty & Spa', 'Buildings and Entertainment Centers', 'Dining', 'Groceries','Home Improvements & Furnishings','Hotels & Casinos', 'Leisure','Medical & Health','Shopping Centers', 'Shops & Services']

In [ ]:
for category in poi_categories:
    long_df[f'category_{category.replace(" ", "_").replace("&", "and")}'] = (long_df['category'] == category).astype(int)

In [ ]:
model_distance = smf.ols('visits ~ (groupbinary) : (timebinary) :(distance_group_0_100) + (groupbinary) : (timebinary) : (distance_group_100_200) + (groupbinary) : (timebinary) : (distance_group_200_300)+ (groupbinary) : (timebinary):(distance_group_300_400) + C(year) + C(poi_id)', 
                data=long_df).fit()
print(model_distance.summary())

In [ ]:
model_cat = smf.ols('visits ~ (groupbinary) : (timebinary) :(category_Apparel) + (groupbinary) : (timebinary) : (category_Beauty_and_Spa) + (groupbinary) : (timebinary) : (category_Buildings_and_Entertainment_Centers)+ (groupbinary) : (timebinary):(category_Dining)+ (groupbinary) : (timebinary) :(category_Groceries) + (groupbinary) : (timebinary) :(category_Home_Improvements_and_Furnishings)  + (groupbinary) : (timebinary) :(category_Hotels_and_Casinos)  + (groupbinary) : (timebinary) :(category_Leisure)  + (groupbinary) : (timebinary) :(category_Medical_and_Health)  + (groupbinary) : (timebinary) :(category_Shopping_Centers)  + (groupbinary) : (timebinary) :(category_Shops_and_Services)   + C(year) + C(poi_id) + C(category) + C(city)', 
                data=long_df).fit()
print(model_cat.summary())